In [ ]:
import os
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle as pkl

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline


In [ ]:
convos = pkl.load(open("messages.pkl", 'rb'))
print("Total Messages: ", len(convos), "\n")


In [ ]:
potentials = []
for i in range(len(convos)):
    if "Alec" in convos[i][0]:
        print(convos[i][1])
